## 13.Transformer组件

**学习目标**

1. 能用代码实现编码器、解码器、编码器-解码器结构

2. 能用代码实现基于正弦余弦函数的位置编码

3. 能用代码实现基于位置的前馈网络（FFN）

4. 能用代码实现残差连接和层规范化(Add&Norm)结构

****

Transformer 是一种深度学习模型，由 Vaswani 等人在 2017 年的论文《Attention Is All You Need》中首次提出。它在自然语言处理（NLP）领域取得了革命性的进展。以下是 Transformer 模型的几个关键特点：

自注意力机制（Self-Attention）：Transformer 摒弃了传统的循环神经网络（RNN）结构，使用自注意力机制来捕捉序列内不同位置之间的依赖关系。这种机制允许模型同时处理序列中的所有元素，从而提高了计算效率。

并行处理能力：由于自注意力机制的特性，Transformer 可以并行处理序列中的所有元素，这与传统的序列模型（如 LSTM 或 GRU）相比，大大提高了训练速度。

Transformer的编码器-解码器架构

Transformer 模型通常由编码器（Encoder）和解码器（Decoder）组成。编码器将输入序列转换为连续的表示，而解码器则使用这些表示来生成输出序列。

多头注意力（Multi-Head Attention）：Transformer 通过多头注意力机制，允许模型同时从不同的表示子空间捕捉信息，增强了模型的表达能力。

位置编码（Positional Encoding）：由于 Transformer 模型本身不具备捕捉序列顺序的能力，因此引入了位置编码来提供序列中每个元素的位置信息。

层归一化（Layer Normalization）和残差连接（Residual Connections）：Transformer 使用层归一化和残差连接来促进深层网络的训练，防止梯度消失或爆炸问题。

<img src="./images/transformer.jpg" style="zoom:60%;" />


预训练和微调：Transformer 模型通常在大量数据上进行预训练，学习通用的语言表示，然后可以在特定任务上进行微调。

广泛的应用：Transformer 不仅在机器翻译上表现出色，还被广泛应用于其他 NLP 任务，如文本摘要、问答系统、文本分类等。

变体和发展：自原始 Transformer 提出以来，出现了许多改进和变体，如 BERT（Bidirectional Encoder Representations from Transformers）、GPT（Generative Pre-trained Transformer）等，这些模型在不同的任务上都取得了显著的成果。

Transformer 模型的提出，标志着 NLP 领域的一个重要转折点，它为处理序列数据提供了一种全新的视角和强大的工具。

****

In [1]:
import math
import torch
import pandas as pd
from torch import nn
from matplotlib import pyplot as plt

1.位置编码

由于Transformer模型本身不具有处理序列顺序的能力，位置编码使得模型能够理解单词在句子中的相对位置。

我们可以使用不同频率的正弦和余弦函数为序列中的每个位置生成唯一的编码。位置编码的公式如下：

<img src="./images/PE.jpg" style="zoom:100%;" />

其中：

𝑃𝐸是位置编码矩阵。

pos是词在序列中的绝对位置（从0开始）。

𝑖是维度索引（从0开始）。

𝑑model是模型的维度大小。

对于每个维度𝑖，位置编码包含两个值：一个正弦值和一个余弦值，分别对应偶数索引和奇数索引。

位置编码的目的是给模型提供每个词在序列中的相对位置信息，这样模型就可以利用这些信息来理解词与词之间的关系。位置编码通常被添加到词嵌入（Word Embeddings）中，然后一起输入到Transformer模型中。

Transformer的位置编码选择三角函数的官方解释是：

位置编码的每个维度都对应于一个正弦曲线。波长形成一个从2π到10000·2π的几何轨迹。我们之所以选择这个函数，是因为我们假设它可以让模型很容易地通过相对位置进行学习，因为对于任何固定的偏移量k,PEpos+k都可以表示为PEpos的线性函数。

也就是说，每个维度都是波长不同的正弦波，波长范围是2π到10000·2π，选用10000这个比较大的数是因为三角函数式有周期的，在这个范围基本上，就不会出现波长一样的情况了。然后谷歌的科学家们为了让PE值的周期更长，还交替使用sin/cos来计算PE的值，就得到了最终的公式。

In [2]:
# 位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, num_hiddens, dropout, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        # 初始化一个全0的位置编码矩阵，大小为(1, max_len, num_hiddens)
        self.P = torch.zeros((1, max_len, num_hiddens))
        # 计算位置编码
        X = torch.arange(max_len, dtype=torch.float32).reshape(-1, 1) / torch.pow(
            10000, torch.arange(0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        # 使用正弦函数为0,2,4,...维度的位置上填充位置编码
        self.P[:, :, 0::2] = torch.sin(X)
        # 使用余弦函数为1,3,5,...维度的位置上填充位置编码
        self.P[:, :, 1::2] = torch.cos(X)
        
    def forward(self, X):
        # 将位置编码加到输入X上
        X += self.P[:, :X.shape[1], :].to(X.device)
        
        return self.dropout(X)

2.基于位置的前馈网络（FFN）

In [3]:
# 基于位置的前馈网络
class PositionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)
        
    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

3.残差连接和层规范化(Add&Norm)

In [4]:
# 残差连接和层规范化(Add&Norm)
class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
        
    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

4.编码器-解码器结构

4.1 编码器

在编码器接口中，我们只指定长度可变的序列作为编码器的输入X。任何继承这个Encoder基类的模型将完成代码实现。

In [5]:
class Encoder(nn.Module):
    """The base encoder interface for the encoder--decoder architecture.
    """
    def __init__(self):
        super().__init__()

    def forward(self, X, *args):
        raise NotImplementedError

4.2 解码器

在下面的解码器接口中，我们新增一个init_state函数，用于将编码器的输出（enc_outputs）转换为编码后的状态。为了逐个地生成长度可变的词元序列，解码器在每个时间步都会将输入和编码后的状态映射成当前时间步的输出词元。

In [6]:
class Decoder(nn.Module):
    def __init__(self):
        # 初始化方法，在这里可以初始化解码器的参数和层
        super().__init__()

    # init_state方法用于初始化解码器的状态。
    # 这个方法应该在每个具体的解码器类中被实现（override）。
    # enc_all_outputs参数代表编码器的所有输出，可以用于初始化解码器的状态。
    # *args允许传入额外的参数，这提供了灵活性，以适应不同的解码器实现。
    def init_state(self, enc_all_outputs, *args):
        raise NotImplementedError

    # X代表输入到解码器的序列数据。
    # state代表解码器的状态，它可能包含编码器的输出、隐藏状态等信息。
    def forward(self, X, state):
        raise NotImplementedError

4.3 编码器-解码器结构

编码器‐解码器结构包含了一个编码器和一个解码器，并且还拥有可选的额外的参数。在前向传播中，编码器的输出用于生成编码状态，这个状态又被解码器作为其输入的一部分。

In [7]:
class EncoderDecoder(nn.Module):
    """编码器-解码器架构的基类"""
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)